In [ ]:
import numpy as np
import pandas as pd

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)

In [ ]:
from datetime import datetime

print(datetime.now().strftime("%d.%m.%Y, %H:%M:%S Uhr"))

In [ ]:
from DateProvider import DateProvider
dateProvider = DateProvider()
print('          lastUpdated:', dateProvider.getLastUpdated())
print('lastUpdatedDataSource:', dateProvider.getLastUpdatedDataSource())        
needsUpdate = dateProvider.needsUpdate()
print('needsUpdate:', needsUpdate)

In [ ]:
def getWorkingDirectory():
    pwd = ! "pwd"
    return pwd[0]

In [ ]:
pwd = getWorkingDirectory()

In [ ]:
pwd

## Download VAERS-Data

In [ ]:
from VAERSFileDownloader import downloadVAERSFile
from zipUtils import unzipAndRemove

def downloadVAERSFileAndUnzip(file):
    downloadedFile = downloadVAERSFile(file, getWorkingDirectory() + "/VAERS/tmp")
    unzipAndRemove(
        zipFile = downloadedFile,
        dstDir = getWorkingDirectory() + '/VAERS/')


In [ ]:
if needsUpdate:
    downloadVAERSFileAndUnzip('2022VAERSData.zip')
    downloadVAERSFileAndUnzip('NonDomesticVAERSData.zip')

In [ ]:
from DataFrameFilter import DataFrameFilter

In [ ]:
from BatchCodeTableFactory import BatchCodeTableFactory

In [ ]:
from HtmlTransformerUtil import HtmlTransformerUtil

In [ ]:
from CountryOptionsSetter import CountryOptionsSetter

In [ ]:
from bs4 import BeautifulSoup


def saveCountryOptions(countryOptions):
    HtmlTransformerUtil().applySoupTransformerToFile(
        file = "../docs/batchCodeTable.html",
        soupTransformer =
            lambda soup:
                BeautifulSoup(
                    CountryOptionsSetter().setCountryOptions(html = str(soup), options = countryOptions),
                    'lxml'))


In [ ]:
def saveLastUpdatedBatchCodeTable(lastUpdated):
    def setLastUpdated(soup):
        soup.find(id = "last_updated").string.replace_with(lastUpdated.strftime(DateProvider.DATE_FORMAT))
        return soup

    HtmlTransformerUtil().applySoupTransformerToFile(
        file = "../docs/batchCodeTable.html",
        soupTransformer = setLastUpdated)

In [ ]:
from IOUtils import IOUtils

In [ ]:
from VaersReader import getVaersForYears, getNonDomesticVaers

In [ ]:
internationalVaers = pd.concat([getVaersForYears([2020, 2021, 2022]), getNonDomesticVaers()])
internationalVaersCovid19 = DataFrameFilter().filterByCovid19(internationalVaers)
internationalVaersCovid19

In [ ]:
def createAndSaveBatchCodeTableForCountry(createBatchCodeTableForCountry, country, minADRsForLethality = None):
    batchCodeTable = createBatchCodeTableForCountry(country)
    batchCodeTable.index.set_names("Batch", inplace = True)
    if minADRsForLethality is not None:
        batchCodeTable.loc[batchCodeTable['Adverse Reaction Reports'] < minADRsForLethality, ['Severe reports', 'Lethality']] = [np.nan, np.nan]
    IOUtils.saveDataFrame(batchCodeTable, '../docs/data/batchCodeTables/' + country)
    # display(country + ":", batchCodeTable)
    display(country)

def createAndSaveBatchCodeTablesForCountries(createBatchCodeTableForCountry, countries, minADRsForLethality = None):
    for country in countries:
        createAndSaveBatchCodeTableForCountry(createBatchCodeTableForCountry, country, minADRsForLethality)

In [ ]:
def getCountryOptions(countries):
    return [getCountryOption(country) for country in countries]

def getCountryOption(country):
    return '<option value="{country}">{country}</option>'.format(country = country)

In [ ]:
countries = sorted(internationalVaersCovid19['COUNTRY'].unique())
countryOptions = ['<option value="Global" selected>Global</option>']  + getCountryOptions(countries)

In [ ]:
saveCountryOptions(countryOptions)

In [ ]:
saveLastUpdatedBatchCodeTable(dateProvider.getLastUpdatedDataSource())

In [ ]:
minADRsForLethality = 100
batchCodeTableFactory = BatchCodeTableFactory(internationalVaersCovid19)

createAndSaveBatchCodeTablesForCountries(
    createBatchCodeTableForCountry = lambda country: batchCodeTableFactory.createBatchCodeTableByCountry(country),
    countries = countries,
    minADRsForLethality = minADRsForLethality)

createAndSaveBatchCodeTableForCountry(
    createBatchCodeTableForCountry = lambda country: batchCodeTableFactory.createGlobalBatchCodeTable(),
    country = 'Global',
    minADRsForLethality = minADRsForLethality)

In [ ]:
def publishGitHubPages():
    %cd /home/frankknoll/Dokumente/Corona/projects/HowBadIsMyBatch-pages
    ! git add -A
    ! git commit -m "updating data for batch codes"
    ! git push

### see https://knollfrank.github.io/HowBadIsMyBatch/batchCodeTable.html

In [ ]:
publishGitHubPages()